In [1]:
# importing modules and packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from io_utils import *
from plotting_utils import *
from analysis_utils import * 
from pathlib import Path
import pickle
import scipy

import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(0)

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import ssm
import seaborn as sns
sns.set_context("talk")

/Users/lencacuturela/opt/anaconda3/envs/ssm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
animalsAgg = ['29L','3095','3096','3097','30B','30L','30R2','4013','4014','4015','4016','91R2'] # list of all aniamls
animalsObs = ['29L','30R2','86L', '87L2'] # list of observer animals
animalsToy = ['86L2', '87B', '87L','87R2']
# sessions = ['t1','t2','t3'] # session in each day
animalsAll = animalsAgg + animalsObs + animalsToy

maxRegions = ['PrL (E)', 'PrL (I)', 'vLS (E)', 'vLS (I)', 'POA (E)', 'POA (I)', 'BNST (E)', 'BNST (I)', 'AH (E)', 'AH (I)', 'MeA (E)', 'MeA (I)', 'VMH (E)', 'VMH (I)', 'PAG (E)', 'PMv (E)', 'PMv (I)', 'LHb (E)', 'LHb (I)', 'PA (E)', 'PA (I)', 'NAc (DA)']

In [3]:
animal = '29L'
group = 'agg'
df = load_and_wrangle(mouseId=animal, group=group, path=None, overwrite=False) # load csv for animal
print(df.head())

regions = get_regions_dataframe(df)
print(regions)

  subject group  other day trial   PrL (E)   PrL (I)   vLS (E)   vLS (I)  \
0     29L   agg  balbc  d1    t1 -0.473779 -1.290629 -0.969278 -0.829421   
1     29L   agg  balbc  d1    t1  0.018012 -1.041894 -0.908416 -0.822654   
2     29L   agg  balbc  d1    t1  0.446211 -0.816871 -0.816333 -0.800908   
3     29L   agg  balbc  d1    t1  0.814519 -0.614451 -0.697274 -0.765899   
4     29L   agg  balbc  d1    t1  1.126641 -0.433526 -0.555484 -0.719340   

    POA (E)  ...   PAG (E)   PMv (E)   PMv (I)   LHb (E)   LHb (I)    PA (E)  \
0  0.341944  ...  1.073387 -0.613695 -1.388937  0.256262 -0.987975 -1.117363   
1  0.326356  ...  0.897360 -0.644853 -1.369702  0.338816 -0.888924 -1.159866   
2  0.302515  ...  0.754962 -0.669580 -1.353668  0.399855 -0.799517 -1.197428   
3  0.271282  ...  0.643002 -0.688342 -1.340514  0.441211 -0.719355 -1.230166   
4  0.233514  ...  0.558292 -0.701603 -1.329918  0.464721 -0.648041 -1.258193   

     PA (I)  NAc (DA)  unsupervised labels  attack labels  
0 

In [4]:
# Set the parameters of the HMM
T = 1000   # number of time bins
D = 2      # number of latent dimensions
N = len(regions)     # number of observed dimensions

In [5]:
Y = get_observed_all_Y(animal, group, path=None)
print(Y[-1].shape)

(9488, 22)


In [6]:
lds = ssm.LDS(N, D, M=0, emissions='gaussian', dynamics="gaussian")